# Pip libarary

In [1]:
from openai import OpenAI
from collections import Counter
import os
import json
import re
import numpy as np
from io import StringIO
import random
import math
import ast

# Load data

In [2]:
def load_solomon_data(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    for i, line in enumerate(lines):
        if "VEHICLE" in line:
            vehicle_line = lines[i + 2]
            break

    vehicle_number_line = vehicle_line.split()
    vehicle_number = int(vehicle_number_line[0])
    vehicle_capacity = int(vehicle_number_line[1])
    vehicle_capacitiesLB = [vehicle_capacity] * vehicle_number

    demandLB = []
    time_windows = []
    coords = []
    due_time_0 = 0

    for line in lines[i + 4:]:
        parts = line.split()
        if len(parts) == 7:
            customer_id = int(parts[0])
            x_coord = int(parts[1])
            y_coord = int(parts[2])
            demand = int(parts[3])
            ready_time = int(parts[4])
            due_time = int(parts[5])
            service_time = int(parts[6])

            if customer_id == 0:
                due_time_0 = due_time
            demandLB.append(demand)
            time_windows.append((ready_time, due_time))
            coords.append((x_coord, y_coord))

    return vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords

In [ ]:
def calculate_distance_matrix(coords):
    n = len(coords)
    distance_matrix = [[0] * n for _ in range(n)]
    for i in range(n):
        for j in range(i + 1, n):
            dist = math.sqrt((coords[i][0] - coords[j][0]) ** 2 + (coords[i][1] - coords[j][1]) ** 2)
            distance_matrix[i][j] = dist
            distance_matrix[j][i] = dist
    return distance_matrix

In [ ]:
def calculate_route_distance(routes, distanceMatrix):
    total_distance = 0
    for route in routes:
        route_distance = 0
        for i in range(len(route) - 1):
            route_distance += distanceMatrix[route[i]][route[i + 1]]
        total_distance += route_distance
    return total_distance

In [ ]:
def process_text(input_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()

    processed_lines = []
    for line in lines:
        line = re.sub(r'\s+', ' ', line.strip())

        line = line + '\n'

        if "READY TIME DUE DATE SERVICE TIME" in line:
            line = line.replace('READY TIME DUE DATE SERVICE TIME', 'READY_TIME DUE_DATE SERVICE_TIME')

        processed_lines.append(line)

    processed_text = ''.join(processed_lines)

    return processed_text

# SA

In [ ]:
def calculate_route_load(routes):
    total_load = 0
    for route in routes:
        route_load = sum(demandLB[customer] for customer in route[1:-1])  # 忽略起点和终点
        total_load += route_load
    return total_load

In [ ]:
def calculate_route_time(routes):
    total_time = 0
    for route in routes:
        for i in range(len(route) - 1):
            total_time += time_windows[route[i]][1] - time_windows[route[i]][0]  # 假设 time_windows 是一个二维数组
    return total_time

In [ ]:
def generate_neighbor(solution):
    new_solution = solution.copy()
    vehicle_id = random.randint(0, len(new_solution) - 1)
    if len(new_solution[vehicle_id]) > 3:
        idx1, idx2 = random.sample(range(1, len(new_solution[vehicle_id]) - 1), 2)
        new_solution[vehicle_id][idx1], new_solution[vehicle_id][idx2] = new_solution[vehicle_id][idx2], new_solution[vehicle_id][idx1]
    return new_solution

In [ ]:
def simulated_annealing(initial_solution, initial_temp, cooling_rate, stopping_temp):
    current_solution = initial_solution
    current_cost = calculate_route_distance(current_solution, distanceMatrix)
    best_solution = current_solution
    best_cost = current_cost
    temperature = initial_temp

    while temperature > stopping_temp:
        new_solution = generate_neighbor(current_solution)
        new_cost = calculate_route_distance(new_solution, distanceMatrix)
        cost_diff = new_cost - current_cost
        if cost_diff < 0 or math.exp(-cost_diff / temperature) > random.random():
            current_solution = new_solution
            current_cost = new_cost
            if current_cost < best_cost:
                best_solution = current_solution
                best_cost = current_cost
        temperature *= cooling_rate
    return best_solution, best_cost

In [ ]:
def print_solution(solution):
    output_str = StringIO() 
    total_distance = calculate_route_distance(solution, distanceMatrix)
    total_load = calculate_route_load(solution)
    total_time = calculate_route_time(solution)

    output_str.write(f'Total time of all routes: {total_time}min\n')
    output_str.write(f'Total distance of all routes: {total_distance}m\n')
    output_str.write(f'Total load of all routes: {total_load}lb\n')

    for vehicle_id, route in enumerate(solution):
        output_str.write(f'Route for vehicle {vehicle_id}:\n')
        route_distance = 0
        route_load = 0
        for i in range(len(route) - 1):
            from_node = route[i]
            to_node = route[i + 1]
            route_distance += distanceMatrix[from_node][to_node]
            route_load += demandLB[to_node]
            output_str.write(f' {from_node} -> {to_node} Load({route_load}) Distance({route_distance})\n')
        output_str.write(f'Time of the route: {total_time}min\n')
        output_str.write(f'Distance of the route: {route_distance}m\n')
        output_str.write(f'Load of the route: {route_load}lb\n')

    return output_str.getvalue()

# check output

In [ ]:
def check(data):
    numbers = re.findall(r'\d+', data)

    flattened_data = [int(num) for num in numbers if int(num) != 0]

    total_count = len(flattened_data)
    print(f"Total number of elements (excluding 0): {total_count}")

    counter = Counter(flattened_data)
    duplicates = {key: value for key, value in counter.items() if value > 1}

    print("Repeated numbers and their counts (excluding 0):")
    for num, count in duplicates.items():
        print(f"Number: {num}, Count: {count}")
    
    return total_count, duplicates

# generate output and test

In [ ]:
openai_api_key = "EMPTY" 
openai_api_base = "http://localhost:8000/v1" 

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

# C109

In [ ]:
file_path = '/root/epfs/CVRPTW copy/c109.txt'
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# C208

In [ ]:
file_path = '/root/epfs/CVRPTW copy/c208.txt'
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# R112

In [ ]:
file_path = '/root/epfs/CVRPTW copy/r112.txt'
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# R203

In [ ]:
file_path = '/root/epfs/CVRPTW copy/r203.txt'
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# R207

In [ ]:
file_path = '/root/epfs/CVRPTW copy/r207.txt'
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# R211

In [ ]:
file_path = '/root/epfs/CVRPTW copy/r211.txt'
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# RC107

In [ ]:
file_path = '/root/epfs/CVRPTW copy/rc107.txt'
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# RC202

In [ ]:
file_path = '/root/epfs/CVRPTW copy/rc202.txt' 
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response


# RC203

In [ ]:
file_path = '/root/epfs/CVRPTW copy/rc203.txt' 
vehicle_number, vehicle_capacity, vehicle_capacitiesLB, demandLB, time_windows, coords = load_solomon_data(file_path)

distanceMatrix = calculate_distance_matrix(coords)

print(vehicle_capacitiesLB)
print(demandLB)
print(time_windows)
print(coords)
print(distanceMatrix)

file_content = process_text(file_path)

In [ ]:
import copy
customer_number = 100

i = 0
j = 0
T = 1
all_distance = []
chat_history = []
first_round_history = []

while i < T:
    if i == 0 and j == 0:
        text_input = f"""Data: {file_content}; 
        Data explanation: There is a starting point (CUST NO.==0) and {customer_number} customer points. All constants are integers. VEHICLE NUMBER is {vehicle_number}, which means the maximum number of vehicles that can be dispatched. CAPACITY is {vehicle_capacity}, which means the maximum load of each vehicle. XCOORD and YCOORF are the horizontal and vertical coordinates of the starting point and the customer point. For the convenience of calculation, the distance between nodes is regarded as the transportation cost between nodes. DEMAND is the demand at the node, and the demand at the starting point depot is 0. READY_TIME indicates the earliest start time of the service at the starting point and the customer point. DUE_TIME indicates the service deadline. For the starting point, this value indicates the latest time when all vehicles must return. SERVICE_TIME indicates the longest duration of service at each node.
        The simple restrictions on the initial solution are that the number of running vehicles is less than or equal to {vehicle_number}, that is, at most {vehicle_number}; each running vehicle starts from 0 and eventually returns to 0, that is, the data of each running vehicle in initial_solution should be [0,...,0]; all of 1 to {customer_number} customer points must be served, that means your output must include all of 1 to {customer_number} numbers, and these {customer_number} numbers from 1 to {customer_number} cannot be repeated. 
The output must adhere strictly to the following constraints:
1. The total number of elements (excluding 0) **must be exactly** {customer_number}. There should be no fewer or greater numbers in the output.
2. The list of numbers must include every integer from **1 to {customer_number}**, without any omissions. This means that the sequence **must** contain all of the numbers from 1 through {customer_number}, and no number should be left out.
3. No number from 1 to {customer_number} should appear more than once. **All numbers must be unique**. Repeated numbers are strictly prohibited and must not appear in the output.
4. If any number is repeated, **exclude** it from the output entirely and make sure that the list still includes **all numbers from 1 to {customer_number}** without repetition.
**Important**: The sequence should only contain unique integers from 1 to {customer_number}. If any duplicate is found, the output should be considered invalid and corrected to meet the exact requirements. This means the model must enforce strict uniqueness and must never allow any duplicates in the list. 
"""
    elif j == -1:
        text_input = f"I check your output and get the Total number of elements (excluding 0) is {total_count}, and the duplicates(Repeated numbers and their counts (excluding 0)) is {duplicates}. I hope the Total number of elements (excluding 0) is {customer_number}, and the the duplicates(Repeated numbers and their counts (excluding 0)) is None. So please give me an answer that meets my requirements."
    elif i > 0 and j == 0:
        text_input = f"I used the initial solution you provided as input to the algorithm and the total_distance (best cost) that I got was {all_distance[i-1]}. Is there any initial solution that can get better results? Please continue to generate an initial solution that can get a sufficiently small total_distance."

    prompt = [{"role": "system", "content": "You are an expert in solving Capacitated Vehicle Routing Problem with Time Windows (CVRPTW) problems. You can output the best initial solution based on the CVRPTW data I provided to make total_distance as small as possible. Do not output any text other than the initial solution. The format example is: [[0, xxx,xxx, 0], [0, xxx, xxx,xxx,xxx, xxx, xxx, 0], ...,[0, xxx,xxx, xxx,0]]. I hope you can strictly follow my output format and element requirements."}] 

    if j == -1:
        temp_history = []

        if chat_history:
            prompt.extend(chat_history) 
        else:
            temp_history.extend(first_round_history) 

        temp_history.append({"role": "user", "content": text_input})
        prompt.extend(temp_history)

    else:
        prompt.extend(chat_history) 
        prompt.append({"role": "user", "content": text_input}) 

    response = client.chat.completions.create(
        model="gemma3_27B",
        messages=prompt,
        max_tokens=4096
    )

    text_response = response.choices[0].message.content
    print(f"text_response: {text_response}")

    total_count, duplicates = check(text_response)
    
    if duplicates and total_count > 100 and len(duplicates) == total_count - 100:
        routes = ast.literal_eval(text_response)
        
        number_locations = {}
        for route_idx, route in enumerate(routes):
            for pos_idx, num in enumerate(route):
                if num != 0: 
                    if num not in number_locations:
                        number_locations[num] = []
                    number_locations[num].append((route_idx, pos_idx))

        to_remove = []
        for num, count in duplicates.items():
            locations = number_locations[num]
            random.shuffle(locations)
            to_remove.extend(locations[1:])
 
        excess = total_count - 100
        if len(to_remove) > excess:
            random.shuffle(to_remove)
            to_remove = to_remove[:excess]
      
        new_routes = copy.deepcopy(routes)
        
        for route_idx, pos_idx in sorted(to_remove, reverse=True):
            new_routes[route_idx].pop(pos_idx)
        
        text_response = json.dumps(new_routes)

        if not text_response.endswith(']'):
            text_response += ']]'
        print(f"exchanged text_response :{text_response}")

        total_count, duplicates = check(text_response)
        print(f"After removing duplicates: total_count={total_count}, duplicates={duplicates}")

    if i == 0 and j == 0:
        first_round_history.append({"role": "user", "content": text_input})
        first_round_history.append({"role": "assistant", "content": text_response})

    if total_count == 100 and not duplicates:
        if i == 0:
            chat_history.append({"role": "user", "content": first_round_history[0]["content"]})
        else:
            chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})

        i += 1
        j = 0
        
        best_solution, best_cost = simulated_annealing(
            initial_solution=ast.literal_eval(text_response),
            initial_temp=1000,
            cooling_rate=0.95,
            stopping_temp=1
        )

        output = print_solution(best_solution)
        print(output) 
        all_distance.append(best_cost)

    elif i > 0 and j == 0:
        chat_history.append({"role": "user", "content": text_input})
        chat_history.append({"role": "assistant", "content": text_response})
        j = -1
    
    else:
        j = -1  
        
        if i == 0 and len(first_round_history) >= 2 and first_round_history[1]["role"] == "assistant":
            first_round_history[1]["content"] = text_response 
        elif i > 0 and len(chat_history) >= 1 and chat_history[-1]["role"] == "assistant":
            chat_history[-1]["content"] = text_response
